<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/notebooks/2021_0406onomatopea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 射影行列の視覚化

- 本日のポイント
射影行列を作って，オノマトペ（に限らず，特定の語句の意味ベクトル such as BERT, word2vec から構成される部分）空間へ任意のベクトルを射影するとき，
元のベクトルで比較するのか，それとも，射影された空間内で比較するのが良いのか

一般に 行列 $X$ が $(n,m)$ サイズで $n>m$ である場合，列次元の n 次元ベクトルから射影行列 
$P_n$ を作成し，行の次元 m から $P_m$ を作成します。
すると，射影行列はたかだか $m$ 次元にしかなりません。
word2vec の次元を 200 とすれば，$n>200$ の語彙を使って射影行列を作っても，構成される射影空間の次元は 200 を超えません。
word2vec のどのような語彙から部分空間を構成してもその空間は 200 次元を超えません。

であれば，どんな語彙ベクトルでもその空間に落とし込んでから比較してみるということを行ってみました。


In [ ]:
import os
import sys
import numpy as np
np.set_printoptions(precision=2)  # numpy の表示桁数設定

from gensim.models import KeyedVectors   # word2vec でーた処理のため gensim を使う
from gensim.models import Word2Vec

# ファイルの所在に応じて変更してください
w2v_base = '.'
w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz'

# word2vec データの読み込み
# ローカルディスクから読み込むようになっています。colab でお使いの場合には適宜変更してください
# word2vec の訓練済モデルを入手
!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz
#!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_sgns.bin.gz
#!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid300_win20_neg20_sgns.bin.gz
#!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz

print('#訓練済 word2vec，訓練データは wikipedia 全文  読み込みに時間がかかります...', end="")
#w2v_base = '/Users/asakawa/study/2016wikipedia/'
w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz'
#w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg10_cbow.bin.gz'
#w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_sgns.bin.gz'
#w2v_file = '2017Jul_jawiki-wakati_neologd_hid200_win20_neg10_sgns.bin.gz'
w2v_file = os.path.join(w2v_base, w2v_file)
w2v = KeyedVectors.load_word2vec_format(w2v_file, 
                                        encoding='utf-8', 
                                        unicode_errors='replace',
                                        binary=True) 
print('done')

# Colab では以下の 2 行の行頭の # を削除してから実行してください
!pip install jaconv
!pip install japanize_matplotlib

import pandas as pd
#2021/Jan 近藤先生からいただいたオノマトペ辞典のデータ
#ひとつ下の '日本語オノマトペ辞典4500より.xls' は著作権の問題があり，公にできません。
# そのため Google Colab での解法，ローカルファイルよりアップロードする
from google.colab import files
uploaded = files.upload()  # ここで `日本語オノマトペ辞典4500より.xls` を指定してアップロードする
ccap_base = '.'
#onomatopea_excel = '日本語オノマトペ辞典4500より.xlsx'
onomatopea_excel = '2021-0325日本語オノマトペ辞典4500より.xls'
onmtp2761 = pd.read_excel(os.path.join(ccap_base, onomatopea_excel), sheet_name='2761語')

In [ ]:
import json
import matplotlib.pyplot as plt
#%matplotlib inline

import scipy
#from scipy import stats

#import tqdm
#import termcolor

# ひらがなカタカナ変換用 `pip install jaconv` してください
!pip install jaconv
import jaconv  

# matplotlib の日本語表示
!pip install japanize_matplotlib
import japanize_matplotlib  

In [ ]:
onomatopea = list(set(sorted(onmtp2761['オノマトペ'])))

print('# オノマトペのうち，word2vec に登録があるかどうかを調査')
kana_entries, kata_entries = [], []
count = 0
for word in onomatopea:
    count += 1
    if word in w2v.vocab:
        kana_entries.append(word)

    kata_w = jaconv.hira2kata(word)
    if kata_w in w2v.vocab:
        kata_entries.append(kata_w)
        
entries = kana_entries + kata_entries
print('There are ', len(entries), ' in ', len(onomatopea), ' onomatopea words in word2vec from jawikipedia')
print('総数がオノマトペデータより多いのは，平仮名表記とカタカナ表記と両者で wikipedia に登録があった場合に重複してカウントしているからです。')
print('カタカナ オノマトペ総数:', len(kata_entries))
print('ひらがな オノマトペ総数:', len(kana_entries))


In [ ]:
# 行列の印字桁数の設定
np.set_printoptions(suppress=False, formatter={'float': '{:6.3f}'.format})

def w2v_similarity_matrix(words, w2v=w2v):
    """word2vec を用いて単語間の類似度行列を算出"""
    ret = np.eye(len(words))
    for i, w0 in enumerate(words):
        j = i
        for w1 in words[i+1:]:
            ret[i,j+1] = w2v.similarity(w0,w1)
            ret[j+1,i] = ret[i,j+1]
            j += 1
        
    return ret


def Mat_orth(X, dtype=np.float):
    """Return Orthogonal Projection and its complement matrices of X
    直交射影行列とその補空間への射影行列を返す。
    X を (n行 m列)としたとき， (m,m) 型の逆行列を算出。(n,n)型ではないことに注意
    
    引数
    X: np.array (n,m)
       入力行列
    戻り値
    Pn: np.array (n,n)
        入力行列の行次元に対応した射影行列 $P_n = X(X^t X)^{-1} X^t$
    Qn: np.arrary(n,n)
        Pn の 直交補空間への射影行列 $Q_n = I_n - P_n$
    Pm: np.array(m,m)
        入力行列の列次元に対応した射影行列 $P_m = X^t (X X^t)^{-1} X$
    Qm: np.array(m,m)
        Pm の直交補空間への射影行列 $Q_m = I_m - P_m$

    """
    n, m = X.shape
    Xt = X.T
    X_Xt, Xt_X = np.dot(X, Xt), np.dot(Xt, X)
    iXXt, iXtX = np.linalg.inv(X_Xt), np.linalg.inv(Xt_X)
    In, Im = np.eye((n),dtype=dtype), np.eye((m), dtype=dtype)
    Pn, Pm = np.dot(X, iXtX), np.dot(Xt, iXXt)
    Pn, Pm = np.dot(Pn, Xt), np.dot(Pm, X)
    Qn, Qm = In - Pn, Im - Pm
    return Pn, Qn, Pm, Qm


def w2vMat(wordlist=['イヌ','ネコ', 'トラ', 'ライオン'], w2v=w2v):
    """len(wordlist)行，word2vec 次元数 列を持つ行列 を返す"""

    if w2v == None:
        assert('Set a `gensim.models.keyedvectors.Word2VecKeyedVectors` as an w2v argument')
        
    # 行列の確保
    X = np.zeros((len(wordlist), w2v.vector_size), dtype=np.float)
        
    # 各行に word2vec ベクトルをコピー
    for i, word in enumerate(wordlist):
        X[i] = np.copy(w2v[word])
            
    return X


def _similarity_matrix(X):
    """行列 X の 各行ベクトルから距離行列を算出して返す"""
    ret = np.eye(X.shape[0])
    X_ = np.copy(X)
    for i in range(X_.shape[0]):
        X_[i] -= X_[i].mean()
        X_[i] /= np.sqrt(X_[i].var())
        
    for i in range(X_.shape[0]):
        for j in range(X_.shape[0]-i):
            #print(i,j+i)
            ret[j+i][i] = np.dot(X_[j+i],X_[i]) / (np.linalg.norm(X_[j+i]) * np.linalg.norm(X_[i]))
            ret[i][j+i] = ret[j+i][i]
    return ret


In [ ]:
from sklearn.decomposition import PCA
#from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


inches = 5
def ax_scatter_gram(ax, pca1, pca2, wordlist, title=None):
    ax.scatter(pca1[:1], pca2[:1], s=200, color='red')
    ax.scatter(pca1[1:], pca2[1:], s=200, color='cyan')
    for i, label in enumerate(wordlist):
        ax.annotate(label, (pca1[i], pca2[i]),fontsize=14)
    ax.set_xlabel("第一主成分")
    ax.set_ylabel("第二主成分")
    ax.set_title(title,fontsize=18)


def compare3plots(wordlist, whole_words, inches=4):

    def plot_pca(ax, R, i, title=""):
        pca = PCA(n_components=2)
        pca_result = pca.fit_transform(R)
        pca1, pca2 = pca_result[:,0], pca_result[:,1] 
        print('\tExplained variation per principal component: {}'.format(pca.explained_variance_ratio_))
        ax_scatter_gram(ax, pca1, pca2, wordlist, title=title)

    print(wordlist)
    fig = plt.figure(figsize=(inches * 3.3,inches))
    ax = fig.add_subplot(1,3,1)
    fontsize = 10

    Own_space = w2vMat(wordlist)
    Pn, Qn, Pm_onmtp, Qm = Mat_orth(w2vMat(wordlist=whole_words))
    y_onmtp = np.dot(Own_space, Pm_onmtp)
    R = _similarity_matrix(y_onmtp)
    #print(R)
    Sim_sorted = np.argsort(R[0])[::-1]  #; print(Sim_sorted)
    print('オノマトペ空間への射影:',[wordlist[s] for s in Sim_sorted])
    ax = fig.add_subplot(1,3,1)
    plot_pca(ax, R, 1, title='オノマトペ空間への射影:')

    Pn, Qn, Pm_inn, Qm = Mat_orth(w2vMat(wordlist=wordlist))
    y_inn = np.dot(Own_space, Pm_inn)
    R = _similarity_matrix(y_inn)
    Sim_sorted = np.argsort(R[0])[::-1]  #; print(Sim_sorted)
    print('構成単語空間への射影:', [wordlist[s] for s in Sim_sorted])
    #print(R)
    ax = fig.add_subplot(1,3,2)
    plot_pca(ax, R, 2, title='構成単語空間への射影:')

    R = _similarity_matrix(Own_space)
    #print(R)
    Sim_sorted = np.argsort(R[0])[::-1] #; print(Sim_sorted)
    print('Original word2vec', [wordlist[s] for s in Sim_sorted])
    ax = fig.add_subplot(1,3,3)
    plot_pca(ax, R, 3, title='Original word2vec')
    
    
    plt.show()
    
#compare3plots(wordlist, entries)
#wordlist = np.random.choice(entries,4)
#compare3plots(wordlist, entries)


wordlist=['イヌ', 'ワンワン','キャンキャン', 'ニャーニャー', 'しくしく']
compare3plots(wordlist,entries)

for N in range(8,12):
    wordlist = np.random.choice(entries,N)
    compare3plots(wordlist,entries)
   


---

以下のセルはオノマトペではありません。
ですがヒントになりそうですので，提示します。
「ある目標語は，続いて提示される 4つの選択肢の中のどれが一番近いか？もっとも適切なものを選べ」という設問があったとき，
４つの選択肢で構成される空間に，目標語を射影し，射影空間内で距離を測るということを行いました。


In [ ]:
Pn, Qn, Pm, Qm = Mat_orth(w2vMat(wordlist=entries))

test_list = [
    ['黒板', '学校', '郵便局', '工場', 'デパート'],
    ['オートバイ', 'ヘルメット', '帽子', '麦わら帽子', '兜'],
    ['柿', '五重塔', '駅', '教会', '病院'],
    ['鹿', '大仏', '東京タワー', '原爆ドーム', '時計台'],
    ['鹿', '奈良', '東京', '広島', '札幌'],
    ['象', 'ピエロ', 'バレエ', '漫才', '歌手'],
    ['急須', '湯呑み', '杯', 'コーヒーカップ', 'コップ'],
#    ['猪', 'うさぎ', 'カンガルー', 'パンダ', 'キリン'],
#    ['そろばん', '電卓', '電池', 'スライド', 'ビデオ'],
#    ['かもめ', '海', '滝', '水田', '池'],
#    ['汽車', 'バス', 'トラック', 'ダンプカー', 'パトカー'],
#    ['急須', '湯呑み', '杯', 'コーヒーカップ', 'コップ'],
#    ['猿', 'ゴリラ', '猫', 'コアラ', '熊'],
]

inches = 5
def ax_scatter_gram(ax, pca1, pca2, wordlist, title=None):
    ax.scatter(pca1[:1], pca2[:1], s=200, color='red')
    ax.scatter(pca1[1:], pca2[1:], s=200, color='cyan')
    for i, label in enumerate(wordlist):
        ax.annotate(label, (pca1[i], pca2[i]),fontsize=14)
    ax.set_xlabel("第一主成分")
    ax.set_ylabel("第二主成分")
    ax.set_title(title,fontsize=18)
    
for i, x in enumerate(test_list):
    wordlist = x
    print('{0:02d} {1}'.format(i,wordlist))

    X = w2vMat(wordlist)
    Pn, Qn, Pm, Qm = Mat_orth(X[1:,])
    Y = np.dot(X, Pm)
    R = _similarity_matrix(Y)
    print('\t射影ベクトルによる類似度行列\n', R)
    
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(R)
    pca1, pca2 = pca_result[:,0], pca_result[:,1] 
    print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

    fig = plt.figure(figsize=(inches * 2,inches))
    ax = fig.add_subplot(1,2,1)
    ax_scatter_gram(ax, pca1, pca2, wordlist, title='選択肢単語空間への射影')

    print('\tword2vec オリジナルの類似度行列\n', w2v_similarity_matrix(wordlist))
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(X)
    pca1, pca2 = pca_result[:,0], pca_result[:,1] 
    print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

    ax = fig.add_subplot(1,2,2)
    ax_scatter_gram(ax, pca1, pca2, wordlist, title="word2vec単語空間への射影")
    plt.show()

sys.exit()

for i, stim_words in enumerate(test_list):
    
    print('{0:02d} {1}'.format(i+1,stim_words))
    sim_mat = w2v_similarity_matrix(stim_words)
    print(sim_mat)
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(sim_mat)
    pca1, pca2 = pca_result[:,0], pca_result[:,1] 
    print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

    plt.figure(figsize=(inches,inches))
    plt.scatter(pca1, pca2, s=200, color='cyan')
    for i, label in enumerate(stim_words):
        plt.annotate(label, (pca1[i], pca2[i]), fontsize=14)
    plt.title('各単語の主成分プロット')
    plt.show()



In [ ]:
def plot_test(wordlist, Pmat=Pm, inches=5):
    #print('{0:02d} {1}'.format(i,wordlist))
    X = w2vMat(wordlist)
    #Pn, Qn, Pm, Qm = Mat_orth(X[1:,])
    Y = np.dot(X, Pmat)
    R = _similarity_matrix(Y)
    print('\t射影ベクトルによる類似度行列\n', R)
    
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(R)
    pca1, pca2 = pca_result[:,0], pca_result[:,1] 
    print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

    fig = plt.figure(figsize=(inches * 2,inches))
    ax = fig.add_subplot(1,2,1)
    ax_scatter_gram(ax, pca1, pca2, wordlist, title='選択肢単語空間への射影')

    print('\tword2vec オリジナルの類似度行列\n', w2v_similarity_matrix(wordlist))
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(X)
    pca1, pca2 = pca_result[:,0], pca_result[:,1] 
    print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

    ax = fig.add_subplot(1,2,2)
    ax_scatter_gram(ax, pca1, pca2, wordlist, title="word2vec単語空間への射影")
    plt.show()


print('#オノマトペ語辞書から N=5 個の単語を選んで，オノマトペ空間へ射影してみる。')
print('#次元が縮約しているので射影した後での空間附置が異なることに注意')
N=5
test_list = np.random.choice(entries, N)
print(test_list)
plot_test(test_list)

---

ここから下が本番です。
任意の単語リストを作成し，作成したリストの一番先頭の要素が，それ以外の要素との距離を比較します。
例えば，`wordlist = ['イヌ', 'ネコ', '自転車', '弁護士', 'ワーワー']` の要素があった場合，
word2vec 空間内での距離とオノマトペ空間内での距離が変化するか否かを検討します。


In [ ]:
from sklearn.decomposition import PCA
#from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


inches = 5
def ax_scatter_gram(ax, pca1, pca2, wordlist, title=None):
    ax.scatter(pca1[:1], pca2[:1], s=200, color='red')
    ax.scatter(pca1[1:], pca2[1:], s=200, color='cyan')
    for i, label in enumerate(wordlist):
        ax.annotate(label, (pca1[i], pca2[i]),fontsize=14)
    ax.set_xlabel("第一主成分")
    ax.set_ylabel("第二主成分")
    ax.set_title(title,fontsize=18)

def compare3plots(wordlist, whole_words, inches=4):

    def plot_pca(ax, R, i, title=""):
        pca = PCA(n_components=2)
        pca_result = pca.fit_transform(R)
        pca1, pca2 = pca_result[:,0], pca_result[:,1] 
        print('\tExplained variation per principal component: {}'.format(pca.explained_variance_ratio_))
        ax_scatter_gram(ax, pca1, pca2, wordlist, title=title)

    print(wordlist)
    fig = plt.figure(figsize=(inches * 3.3,inches))
    fontsize = 10

    Own_space = w2vMat(wordlist)
    Pn, Qn, Pm_onmtp, Qm = Mat_orth(w2vMat(wordlist=whole_words))
    y_onmtp = np.dot(Own_space, Pm_onmtp)
    R = _similarity_matrix(y_onmtp)
    #print(R)
    Sim_sorted = np.argsort(R[0])[::-1]  #; print(Sim_sorted)
    print('オノマトペ空間への射影:',[wordlist[s] for s in Sim_sorted], end=" ")
    ax = fig.add_subplot(1,3,1)
    plot_pca(ax, R, 1, title='オノマトペ空間への射影:')

    Pn, Qn, Pm_inn, Qm = Mat_orth(w2vMat(wordlist=wordlist))
    y_inn = np.dot(Own_space, Pm_inn)
    R = _similarity_matrix(y_inn)
    Sim_sorted = np.argsort(R[0])[::-1]  #; print(Sim_sorted)
    print('構成単語空間への射影:', [wordlist[s] for s in Sim_sorted], end=" ")
    #print(R)
    ax = fig.add_subplot(1,3,2)
    plot_pca(ax, R, 2, title='構成単語空間への射影:')

    R = _similarity_matrix(Own_space)
    #print(R)
    Sim_sorted = np.argsort(R[0])[::-1] #; print(Sim_sorted)
    print('Original word2vec', [wordlist[s] for s in Sim_sorted], end=" ")
    ax = fig.add_subplot(1,3,3)
    plot_pca(ax, R, 3, title='Original word2vec')
    
    
    plt.show()
    
#compare3plots(wordlist, entries)
#wordlist = np.random.choice(entries,4)
#compare3plots(wordlist, entries)


#wordlist=['イヌ', 'ワンワン','キャンキャン', 'ニャーニャー', 'しくしく']
wordlist=['イヌ', 'パイナップル', '自転車', '弁護士', 'キャンキャン']
compare3plots(wordlist,entries)

for N in range(8,12):
    wordlist = np.random.choice(entries,N)
    compare3plots(wordlist,entries)
   


